In [1]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("shusrith/machine-trainslation")

# print("Path to dataset files:", path)

In [2]:
import pandas as pd
from sklearn.utils import shuffle

df = pd.read_csv(f"EnglishOrSpanish/output_joint1.csv")
df = shuffle(df)
df

,English,Spanish
117633,"[2, 6140, 265, 2914, 458, 290, 1452, 2145, 265...","[2, 3937, 410, 6575, 346, 290, 302, 620, 410, ..."
128829,"[2, 1972, 506, 1154, 304, 272, 3279, 768, 588,...","[2, 962, 375, 311, 6933, 316, 4021, 294, 1986,..."
127698,"[2, 2131, 414, 465, 497, 705, 437, 3351, 270, ...","[2, 669, 302, 11683, 573, 757, 20053, 14335, 2..."
48804,"[2, 270, 411, 437, 3342, 425, 1807, 224, 3, 0,...","[2, 616, 355, 1543, 1953, 1836, 224, 3, 0, 0, ..."
18518,"[2, 270, 4715, 290, 1496, 582, 224, 3, 0, 0, 0...","[2, 372, 2498, 782, 265, 290, 224, 3, 0, 0, 0,..."
...,...,...
31662,"[2, 878, 3645, 300, 1817, 224, 3, 0, 0, 0, 0, ...","[2, 331, 895, 12970, 331, 1423, 224, 3, 0, 0, ..."
80887,"[2, 290, 588, 3520, 300, 1961, 2196, 224, 3, 0...","[2, 290, 302, 772, 6866, 500, 321, 882, 224, 3..."
27680,"[2, 3351, 334, 402, 5623, 224, 3, 0, 0, 0, 0, ...","[2, 2088, 318, 403, 7198, 224, 3, 0, 0, 0, 0, ..."
12260,"[2, 818, 775, 8203, 224, 3, 0, 0, 0, 0, 0, 0, ...","[2, 333, 1289, 12419, 224, 3, 0, 0, 0, 0, 0, 0..."


In [3]:
import ast

df["English"] = df["English"].apply(ast.literal_eval)
df["Spanish"] = df["Spanish"].apply(ast.literal_eval)
df

,English,Spanish
117633,"[2, 6140, 265, 2914, 458, 290, 1452, 2145, 265...","[2, 3937, 410, 6575, 346, 290, 302, 620, 410, ..."
128829,"[2, 1972, 506, 1154, 304, 272, 3279, 768, 588,...","[2, 962, 375, 311, 6933, 316, 4021, 294, 1986,..."
127698,"[2, 2131, 414, 465, 497, 705, 437, 3351, 270, ...","[2, 669, 302, 11683, 573, 757, 20053, 14335, 2..."
48804,"[2, 270, 411, 437, 3342, 425, 1807, 224, 3, 0,...","[2, 616, 355, 1543, 1953, 1836, 224, 3, 0, 0, ..."
18518,"[2, 270, 4715, 290, 1496, 582, 224, 3, 0, 0, 0...","[2, 372, 2498, 782, 265, 290, 224, 3, 0, 0, 0,..."
...,...,...
31662,"[2, 878, 3645, 300, 1817, 224, 3, 0, 0, 0, 0, ...","[2, 331, 895, 12970, 331, 1423, 224, 3, 0, 0, ..."
80887,"[2, 290, 588, 3520, 300, 1961, 2196, 224, 3, 0...","[2, 290, 302, 772, 6866, 500, 321, 882, 224, 3..."
27680,"[2, 3351, 334, 402, 5623, 224, 3, 0, 0, 0, 0, ...","[2, 2088, 318, 403, 7198, 224, 3, 0, 0, 0, 0, ..."
12260,"[2, 818, 775, 8203, 224, 3, 0, 0, 0, 0, 0, 0, ...","[2, 333, 1289, 12419, 224, 3, 0, 0, 0, 0, 0, 0..."


In [4]:
import torch
from torch.utils.data import DataLoader, TensorDataset

train_data = TensorDataset(
    torch.tensor(df["English"][:100000].tolist(), dtype=torch.long),
    torch.tensor(df["Spanish"][:100000].tolist(), dtype=torch.long),
)
test_data = TensorDataset(
    torch.tensor(df["English"][100000:].tolist(), dtype=torch.long),
    torch.tensor(df["Spanish"][100000:].tolist(), dtype=torch.long),
)
train_loader = DataLoader(train_data, batch_size=256, shuffle=True, drop_last=True)
test_loader = DataLoader(test_data, batch_size=256, shuffle=False, drop_last=True)

In [5]:
import json

with open(f"EnglishOrSpanish/vocab.json", "r") as f:
    vocab = json.load(f)

In [6]:
import torch
import torch.nn as nn


class Encoder(nn.Module):
    def __init__(
        self, vocab_size, embedding_dim, hidden_size, num_layers=2, dropout=0.3
    ):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.vocab_size = vocab_size

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(
            embedding_dim,
            hidden_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout,
        )
        self.dropout = nn.Dropout(dropout)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.LSTM):
            for name, param in module.named_parameters():
                if "weight" in name:
                    nn.init.xavier_uniform_(param)
                elif "bias" in name:
                    nn.init.zeros_(param)

    def forward(self, x):
        embedded = self.dropout(self.embedding(x))
        output, (hidden, cell) = self.lstm(embedded)
        return output, hidden, cell

In [7]:
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.hidden_size = hidden_size

        self.W1 = nn.Linear(hidden_size, hidden_size)
        self.W2 = nn.Linear(hidden_size * 2, hidden_size)
        self.v = nn.Linear(hidden_size, 1, bias=False)

    def forward(self, hidden, enc_out):
        hidden = hidden.unsqueeze(1)
        score = self.v(torch.tanh(self.W1(hidden) + self.W2(enc_out)))
        attention_weights = torch.softmax(score, dim=1)
        context_vector = torch.sum(attention_weights * enc_out, dim=1)
        return context_vector, attention_weights

In [8]:
import torch
import torch.nn as nn


class Decoder(nn.Module):
    def __init__(
        self, vocab_size, embedding_dim, hidden_size, num_layers=2, dropout=0.3
    ):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.vocab_size = vocab_size

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)

        self.lstm = nn.LSTM(
            embedding_dim + hidden_size * 2,
            hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
            bidirectional=True,
        )
        self.fc = nn.Linear(hidden_size * 2, vocab_size)
        self.dropout = nn.Dropout(dropout)
        self.attention = Attention(hidden_size)
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.LSTM):
            for name, param in module.named_parameters():
                if "weight" in name:
                    nn.init.xavier_uniform_(param)
                elif "bias" in name:
                    nn.init.zeros_(param)

    def forward(self, x, hidden, cell, enc_out):
        attention_vector, _ = self.attention(hidden[0], enc_out)
        x = self.embedding(x)
        x = torch.cat((attention_vector, x), dim=-1)
        x = x.unsqueeze(1)
        output, (hidden, cell) = self.lstm(x, (hidden, cell))
        output = self.dropout(output)
        output = self.fc(output)
        return output, hidden, cell

In [9]:
import torch.nn as nn
import torch
import random

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, vocab_size, teacher_forcing_ratio=0.5):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.vocab_size = vocab_size
        self.teacher_forcing_ratio = teacher_forcing_ratio

    def forward(self, src, trg):
        enc_out, hidden, cell = self.encoder(src)
        outputs = torch.zeros(trg.shape[0], trg.shape[1], self.decoder.vocab_size).to("cuda")
        x = trg[:, 0]
        for t in range(1, trg.shape[1]):
            output, hidden, cell = self.decoder(x, hidden, cell, enc_out)
            output = output.squeeze(1)
            outputs[:, t, :] = output
            use_teacher_forcing = random.random() < self.teacher_forcing_ratio
            x = trg[:, t] if use_teacher_forcing else output.argmax(dim=1)

        return outputs
    
    def predict(self, src):
        enc_out, hidden, cell = self.encoder(src)
        outputs = torch.zeros(src.shape[0], src.shape[1], self.decoder.vocab_size).to("cuda")
        x = src[:, 0]
        for i in range(1, src.shape[1]):
            output, hidden, cell = self.decoder(x, hidden, cell, enc_out)
            output = output.squeeze(1)
            outputs[:, i, :] = output
            x = output.argmax(dim=1)
        
        return outputs

In [ ]:
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

def train(
        model,
        train_loader,
        test_loader,
        optimizer,
        criterion,
        device,
        scheduler, 
        num_epochs,
    ):
    writer = SummaryWriter()

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        model.train()
        epoch_loss = 0
        progress_bar = tqdm(train_loader, desc="Training", leave=False)

        for src, trg in progress_bar:
            src, trg = src.to(device), trg.to(device)
            optimizer.zero_grad()
            outputs = model(src, trg)
            loss = criterion(
                    outputs[:, 1:].reshape(-1, model.vocab_size), trg[:, 1:].reshape(-1)
                )
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

            progress_bar.set_postfix(
                    loss=f"{loss.item():.4f}"
                )

        train_loss = epoch_loss / len(train_loader)

        # ---------------- VALIDATION ---------------- #
        model.eval()
        val_epoch_loss = 0
        progress_bar = tqdm(test_loader, desc="Validating", leave=False)

        with torch.no_grad():
            for src, trg in progress_bar:
                src, trg = src.to(device), trg.to(device)
                outputs = model(src, trg)
                loss = criterion(
                        outputs[:, 1:].reshape(-1, model.vocab_size),
                        trg[:, 1:].reshape(-1),
                    )
                val_epoch_loss += loss.item()

                progress_bar.set_postfix(
                        loss=f"{loss.item():.4f}"
                    )

        val_loss = val_epoch_loss / len(test_loader)
        scheduler.step(val_loss)

        # Log metrics to TensorBoard
        writer.add_scalar("Loss/Train", train_loss, epoch)
        writer.add_scalar("Loss/Validation", val_loss, epoch)
        writer.add_scalar("Learning Rate", scheduler.get_last_lr()[0], epoch)

        print(
            f"Train loss : {train_loss}, Val_loss : {val_loss}, lr: {scheduler.get_last_lr()[0]}"
        )

    writer.close()
    return train_loss, val_loss

2025-03-23 00:06:16.804905: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-23 00:06:16.826606: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742668576.857692   10912 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742668576.864500   10912 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-23 00:06:16.887164: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

vocab_size = len(vocab)
embedding_dim = 256
hidden_size = 64
encoder = Encoder(vocab_size, embedding_dim, hidden_size).to(device)
decoder = Decoder(vocab_size, embedding_dim, hidden_size).to(device)
seq2seq = Seq2Seq(encoder, decoder, vocab_size).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(seq2seq.parameters(), lr=0.005, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="min", factor=0.5, patience=2
)

In [12]:
import torch
import torch.nn.functional as F
from tqdm import tqdm

train(
    seq2seq,
    train_loader,
    test_loader,
    optimizer,
    criterion,
    device,
    scheduler,
    60,
)


Epoch 1/60


Train loss : 5.398456978186583, Val_loss : 4.6444296077289415, lr: 0.005

Epoch 2/60


Train loss : 4.4553677999056305, Val_loss : 4.00223692328529, lr: 0.005

Epoch 3/60


Train loss : 3.9469068808433336, Val_loss : 3.54923131191625, lr: 0.005

Epoch 4/60


Train loss : 3.557183112242283, Val_loss : 3.2092477220349607, lr: 0.005

Epoch 5/60


...

Train loss : 1.6974349764677195, Val_loss : 2.0174991462082987, lr: 0.000625

Epoch 58/60


Train loss : 1.6741172179197654, Val_loss : 2.012294535088328, lr: 0.0003125

Epoch 59/60


Train loss : 1.6385820404077187, Val_loss : 2.0219949842554277, lr: 0.0003125

Epoch 60/60


Train loss : 1.6478714777873114, Val_loss : 2.018457844194058, lr: 0.0003125


(1.6478714777873114, 2.018457844194058)

In [13]:
torch.save(seq2seq.state_dict(), "seq2seq-bidir-attention.pth")

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

vocab_size = len(vocab)
embedding_dim = 256
hidden_size = 64
encoder = Encoder(vocab_size, embedding_dim, hidden_size).to(device)
decoder = Decoder(vocab_size, embedding_dim, hidden_size).to(device)
seq2seq = Seq2Seq(encoder, decoder, vocab_size).to(device)
seq2seq.load_state_dict(torch.load("seq2seq-bidir-attention.pth", weights_only=True))

<All keys matched successfully>

In [15]:
seq2seq.eval()
# with torch.no_grad():
a, b = next(iter(train_loader))
a = a.to(device)
b = b.to(device)
with torch.no_grad():
    x = seq2seq.predict(a)

x = x.argmax(dim=2)
x.shape

torch.Size([256, 18])

In [16]:
from tokenizers import ByteLevelBPETokenizer
tokenizer = ByteLevelBPETokenizer(
    f"EnglishOrSpanish/vocab.json", f"EnglishOrSpanish/merges.txt"
)

In [17]:
print(tokenizer.decode(b.tolist()[8]))
print(tokenizer.decode(x.tolist()[8]))

<SOS> nunca me dijiste nada sobre el accidente <EOS><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD>
<PAD> nunca me dijiste nada acerca del accidente <EOS><EOS><EOS><EOS><EOS><EOS><EOS><EOS><EOS>
